<a href="https://colab.research.google.com/github/Nautyy9/search_engine/blob/main/Web_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("drive")

Mounted at drive


In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import pandas as pd
import os
import re
import csv
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import AutoTokenizer,create_optimizer,TFAutoModel


In [ ]:
CONFIG = {
    "MAX_LENGTH"  : 64
    "BATCH_SIZE"  : 128
}

In [ ]:
# !pip install aicrowd-cli

In [ ]:
# !aicrowd login

In [ ]:
# !aicrowd dataset download -c esci-challenge-for-improving-product-search

### We only need product_catalogue and train dir out the 4 dir availble to us

In [ ]:
# !unzip "/content/product_catalogue-v0.3.csv.zip" -d "/content/dataset/"

In [ ]:
# !unzip "/content/train-v0.3.csv.zip" -d "/content/dataset/"

In [ ]:
# "/content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv"
# "/content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv"

In [ ]:
# !cp /content/dataset/data/processed/public/task_3_product_substitute_identification/train-v0.3.csv /content/drive/MyDrive/datasets/kdd_cup
# !cp /content/dataset/data/processed/public/task_3_product_substitute_identification/product_catalogue-v0.3.csv /content/drive/MyDrive/datasets/kdd_cup

In [ ]:
train_path  = "/content/drive/MyDrive/datasets/kdd_cup/train-v0.3.csv"
catalogue_path = '/content/drive/MyDrive/datasets/kdd_cup/product_catalogue-v0.3.csv'

In [ ]:
catalogue_data = pd.read_csv(catalogue_path)
train_data = pd.read_csv(train_path)

In [ ]:
train_data.loc[train_data["query_locale"] == "us"].head()

,example_id,query,product_id,query_locale,substitute_label
249721,249721,revent 80 cfm,B000MOO21W,us,no_substitute
249722,249722,revent 80 cfm,B001E6DMKY,us,no_substitute
249723,249723,revent 80 cfm,B003O0MNGC,us,no_substitute
249724,249724,revent 80 cfm,B00MARNO5Y,us,no_substitute
249725,249725,revent 80 cfm,B00XBZFWWM,us,no_substitute


In [ ]:
print(len(catalogue_data))
catalogue_data.head()


1815216


,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),NaN,NaN,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,NaN,NaN,11 Degrees,NaN,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,NaN,NaN,11 Degrees,Negro (,es


In [ ]:
def get_product_title(id):
  id_array = []
  id_array.append(str(id))
  get_product = catalogue_data.loc[catalogue_data["product_id"].isin(id_array)]
  # since we are extracting the return value of cells which gives us the columns name if the result is true
  # get_product gives us the columns from which we select a selected one and see if the value is avail or not , like here the product title is avail or not
  return get_product["product_title"][(int(str(get_product["product_title"]).split("   ")[0]))]
  # wee get a long string containing the id and the text from which we need to extract the id and in order to do that , we need to do split , but since the out is a Series o/p hence we first convert it to string then do split
  # check for the no. of spaces in the split, get the id and extract the value from it

In [ ]:
get_product_title("B000MOO21W")

'Panasonic FV-20VQ3 WhisperCeiling 190 CFM Ceiling Mounted Fan'

# Dataset Preperation

In [ ]:
product_titles = [str(catalogue_data["product_title"][i]) for i in range(len(catalogue_data))]

In [ ]:
product_titles[0]

'11 Degrees de los Hombres Playera con Logo, Negro, L'

In [ ]:
len(product_titles)//10

181521

In [ ]:
yooo = [train_data["query_locale"][i] for i in range(len(train_data))]

In [ ]:
for ind,i in enumerate(yooo):
  if(i != "es"):
    print(ind)
    break
  else:
    pass

249721


In [ ]:
# product_titles_path = "/content/drive/MyDrive/datasets/Sent_trans_product_title"

In [ ]:
# np.save(product_titles_path+"product_titles.npy", product_titles)

In [ ]:
filepaths = ["/content/drive/MyDrive/datasets/kdd_cup/csv_file"+str(i)+".csv" for i in range(10)]
print(filepaths)

['/content/drive/MyDrive/datasets/kdd_cup/csv_file0.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file1.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file2.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file3.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file4.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file5.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file6.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file7.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file8.csv', '/content/drive/MyDrive/datasets/kdd_cup/csv_file9.csv']


In [ ]:

header = ['query', 'product',]
for index, files in enumerate(filepaths):
  with open(files, 'w', encoding='UTF8') as f:

  # create the csv writer
    writer = csv.writer(f)

  # write the header
    writer.writerow(header)
    match index:
      # case 0:
      #   for i in range(0,2000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 1:
      #   for i in range(150000, 152000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 2:
      #   for i in range(300000, 302000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 3:
      #   for i in range(500000, 502000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 4:
      #   for i in range(700000, 702000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 5:
      #   for i in range(900000,902000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 6:
      #   for i in range(1100000,1102000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      # case 7:
      #   for i in range(1400000 , 1402000):
      #     data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
      #     writer.writerow(data)
      case 8:
        for i in range(1600000, 1602000):
          data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
          writer.writerow(data)
      case 9:
        for i in range(1800000, 1802000):
          data=[train_data[i:i+1]['query'][i],get_product_title(train_data[i:i+1]['product_id'][i])]
          writer.writerow(data)


In [ ]:
181521*10

1815210

In [ ]:
csv_dataset = load_dataset("csv", data_files=filepaths)

FileNotFoundError: ignored

In [ ]:
csv_dataset

In [ ]:
# import os
# for file in filepaths:
#   file_path = os.path.join("/content/drive/MyDrive/datasets/kdd_cup", file)
#   with open(file_path, "r"):
#     print(file)

In [ ]:
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
def preprocess(dataset):
  if dataset['product'] == None:
    dataset['product'] = dataset['query']

  dataset['input_ids_query']=[]
  dataset['token_type_ids_query']=[]
  dataset['attention_mask_query']=[]

  dataset['input_ids_product']=[]
  dataset['token_type_ids_product']=[]
  dataset['attention_mask_product']=[]

  tokenized_output_query=tokenizer(dataset['query'],max_length=CONFIG["MAX_LENGTH"],padding='max_length',truncation=True)
  tokenized_output_product=tokenizer(dataset['product'],max_length=CONFIG["MAX_LENGTH"],padding='max_length',truncation=True)

  dataset['input_ids_query'].append(tokenized_output_query['input_ids'])
  dataset['token_type_ids_query'].append(tokenized_output_query['token_type_ids'])
  dataset['attention_mask_query'].append(tokenized_output_query['attention_mask'])

  dataset['input_ids_product'].append(tokenized_output_product['input_ids'])
  dataset['token_type_ids_product'].append(tokenized_output_product['token_type_ids'])
  dataset['attention_mask_product'].append(tokenized_output_product['attention_mask'])

  return dataset

In [ ]:
prep_dataset= csv_dataset.map(preprocess)

In [ ]:
prep_dataset

In [ ]:
tf_dataset = prep_dataset['train'].to_tf_dataset(
    columns= ['input_ids_query', 'token_type_ids_query', 'attention_mask_query','input_ids_product', 'token_type_ids_product', 'attention_mask_product', 'label'],
    shuffle= True,
    batch_size = CONFIG["BATCH_SIZE"],
)

In [ ]:
tf_dataset

In [ ]:
model = TFAutoModel.from_pretrained(model_id)
model.summary()

In [ ]:
class SentenceTransformer(tf.keras.Model):
  def __init__(self, model):
    def super(SentenceTransformer, self).__init__()

    self.model = model
    self.dense = Dense(1, activation="sigmoid")

  def compile(self, optimizer, loss_fn):

    super(SentenceTransformer, self).compile()
    # we are calling super.compile() in the  compile function derived from tf.keras.Model super class ?
    # yes , because this helps in 1) Proper Initialization 2) Inheritance behavior 3) Future Profing

    # 1) Proper Initialization --> compile() method in tf.keras.Model class initializes attributes for the tf.keras.Model setup hence to ensure that these attributes are initialized properly before any customization made by us , is the reason we use super().compile()
    # 2) Inheritance Behavior ---> since we are inheriting from  tf.keras.Model hence we need that every thing inherited works as expected inside the the compile function
    # 3) Future Profing  ---> since we know that libraries and their methods improve or gets update and to accomodate with these updates we have to do super().compile() along with our customization


    self.optimizer = optimizer
    self.loss_fn = loss_fn
    self.metric = tf.keras.metrics.Mean(name="loss")

  @property
  def metrics(self):
    return [self.metric]

  def mean_pooling(self, model_output, attention_mask):
    # taking in the first embedding output of the selected(i.e, [0]th)  sentence
    token_embeddings = model_output[0]
    input_mask = tf.cast(
        tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)), tf.float32
    )
    return tf.math.reduce_sum(token_embeddings * input_mask, axis=1)/tf.clip_by_value(tf.math.reduce_sum(input_mask , axis=1) 1e-9, tf.float32.max)

  def train_step(self, train_data):
    query={'input_ids':train_data['input_ids_query'][:,0,:],
           'token_type_ids':train_data['token_type_ids_query'][:,0,:],
           'attention_mask':train_data['attention_mask_query'][:,0,:]}

    product={'input_ids':train_data['input_ids_product'][:,0,:],
             'token_type_ids':train_data['token_type_ids_product'][:,0,:],
             'attention_mask':train_data['attention_mask_product'][:,0,:]}

    with tf.GradientTape() as recorder:
      query_predictions=self.model(query)
      pred_query=self.mean_pooling(query_predictions,train_data['attention_mask_query'][:,0,:])

      product_predictions=self.model(product)
      pred_product=self.mean_pooling(product_predictions,train_data['attention_mask_product'][:,0,:])

      pred_concat=tf.concat([pred_query,pred_product,tf.abs(pred_query-pred_product)],axis=-1)

      predictions=self.dense(pred_concat)
      loss=self.loss_fn(predictions)


    partial_derivatives = recorder.gradient(loss,self.model.trainable_weights)
    self.optimizer.apply_gradients(zip(partial_derivatives, self.model.trainable_weights))


    self.loss_metric.update_state(loss)

    return {'loss':self.loss_metric.result(),}

In [ ]:
stransformer=SentenceTransformer(model)
stransformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5,),
    loss_fn=tf.keras.losses.BinaryCrossentropy(),
)

In [ ]:
EPOCHS=5
history=stransformer.fit(tf_dataset,epochs=EPOCHS,)

In [ ]:
model_path='/content/drive/MyDrive/stransformer/stransformers.h5'

In [ ]:
model.load_weights(model_path)

In [ ]:
def mean_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]

  input_mask_expand = tf.cast(
      tf.broadcast_to(tf.expand_dims(attention_mask, -1), tf.shape(token_embeddings)),
      tf.float32
  )
  return tf.math.reduce_sum(token_embeddings * input_mask_expand)

In [ ]:
import tensorflow as tf
hits = tf.constant([[1,2,3,4,5] , [12,324,523,3,1]]) # shape (2,5)
shits = tf.broadcast_to(hits, [5  ,2,5])
print(shits)

tf.Tensor(
[[[  1   2   3   4   5]
  [ 12 324 523   3   1]]

 [[  1   2   3   4   5]
  [ 12 324 523   3   1]]

 [[  1   2   3   4   5]
  [ 12 324 523   3   1]]

 [[  1   2   3   4   5]
  [ 12 324 523   3   1]]

 [[  1   2   3   4   5]
  [ 12 324 523   3   1]]], shape=(5, 2, 5), dtype=int32)
